In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

In [3]:
player_ids = pd.read_csv('files/SFBB Player ID Map - PLAYERIDMAP.csv')
player_ids = player_ids[['MLBNAME', 'MLBID']].copy()
display([(i, player_ids[i].dtype) for i in player_ids.columns])
player_ids = player_ids.dropna(subset=['MLBID']).copy()
player_ids['MLBID'] = player_ids['MLBID'].astype('int64')
display([(i, player_ids[i].dtype) for i in player_ids.columns])
batter_names = player_ids.drop_duplicates(subset='MLBID').set_index('MLBID')['MLBNAME']

[('MLBNAME', dtype('O')), ('MLBID', dtype('float64'))]

[('MLBNAME', dtype('O')), ('MLBID', dtype('int64'))]

In [4]:
savant_2024 = pd.read_csv('files/2024/savantdata-2024.csv')

In [5]:
savant_2024['catcher_name'] = savant_2024['fielder_2'].map(batter_names)
print("Missing names:", savant_2024['catcher_name'].isna().sum())
print("Total rows:", len(savant_2024))

Missing names: 16421
Total rows: 708825


In [6]:
savant_2024.loc[:, 'pitch_uid'] = (savant_2024['game_pk'].astype(str) + '_' + savant_2024['at_bat_number'].astype(str) + '_' + savant_2024['pitch_number'].astype(str))
savant_2024 = savant_2024.drop_duplicates(subset=['pitch_uid'])

In [7]:
display(savant_2024.shape)
display(savant_2024.head())

(708825, 120)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,catcher_name,pitch_uid
0,FF,2024-09-30,97.4,-2.10,4.88,"Díaz, Edwin",518595,621242,field_out,hit_into_play,NaN,NaN,NaN,NaN,3.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,X,6.0,ground_ball,2,2,2024,-0.96,0.99,0.67,3.00,NaN,642201.0,NaN,2,9,Bot,112.78,146.97,NaN,NaN,NaN,NaN,9.893410,-141.548522,-1.857108,-15.148286,30.423852,-18.544662,3.43,1.54,5.0,87.6,-30.0,99.9,2196.0,7.6,747139,682626,624413,657193,578428,596019,607043,621438,516782,52.91,0.049,0.052,0.0,1.0,0.0,0.0,2.0,82,5,4-Seam Fastball,7,8,7,8,8,7,7,8,Standard,Strategic,232.0,-0.142,-0.248,68.8,7.3,0.059,0.248,88.0,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,1.40,0.96,0.96,17.6,6.149605,12.090516,22.160400,45.805662,22.048373,Francisco Alvarez,747139_82_5
1,SL,2024-09-30,90.7,-2.14,5.06,"Díaz, Edwin",518595,621242,NaN,ball,NaN,NaN,NaN,NaN,14.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,B,NaN,NaN,1,2,2024,0.20,0.61,0.75,1.20,NaN,642201.0,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,6.952856,-131.970548,-5.159843,1.012613,25.103830,-24.224355,3.47,1.52,NaN,NaN,NaN,92.9,2209.0,7.3,747139,682626,624413,657193,578428,596019,607043,621438,516782,53.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,4,Slider,7,8,7,8,8,7,7,8,Standard,Strategic,201.0,0.000,0.037,NaN,NaN,NaN,-0.037,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.14,-0.20,-0.20,23.1,NaN,NaN,NaN,NaN,NaN,Francisco Alvarez,747139_82_4
2,SL,2024-09-30,91.1,-2.07,5.14,"Díaz, Edwin",518595,621242,NaN,swinging_strike,NaN,NaN,NaN,NaN,9.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,S,NaN,NaN,1,1,2024,0.12,0.35,0.66,1.61,NaN,642201.0,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,6.757602,-132.670539,-3.857177,0.114032,25.108326,-27.443443,3.43,1.54,NaN,NaN,NaN,93.5,2302.0,7.4,747139,682626,624413,657193,578428,596019,607043,621438,516782,53.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,3,Slider,7,8,7,8,8,7,7,8,Standard,Strategic,210.0,0.000,-0.060,71.2,8.9,NaN,0.060,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.37,-0.12,-0.12,22.4,23.541699,-27.093819,34.778701,45.227965,45.368412,Francisco Alvarez,747139_82_3
3,SL,2024-09-30,91.3,-2.05,5.07,"Díaz, Edwin",518595,621242,NaN,ball,NaN,NaN,NaN,NaN,14.0,"Travis d'Arnaud grounds out, shortstop Francis...",R,R,R,ATL,NYM,B,NaN,NaN,0,1,2024,0.21,0.63,0.61,1.18,NaN,642201.0,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,6.385514,-132.826939,-5.397765,1.181254,26.329685,-23.817729,3.47,1.54,NaN,NaN,NaN,93.5,22

In [8]:
display(savant_2024.columns[savant_2024.isna().sum() / len(savant_2024) >= 0.9])
old_system_cols = ['spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire']

savant_2024 = savant_2024.drop(columns = old_system_cols).copy()
savant_2024 = savant_2024.dropna(axis=0, subset=['pitch_type', 'zone']).copy()

Index(['spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated',
       'break_length_deprecated', 'on_3b', 'tfs_deprecated',
       'tfs_zulu_deprecated', 'umpire', 'sv_id'],
      dtype='object')

In [10]:
savant_2024.loc[:, 'batter_name'] = savant_2024['batter'].map(batter_names).astype('object')
savant_2024['batter_name']

0          Travis d'Arnaud
1          Travis d'Arnaud
2          Travis d'Arnaud
3          Travis d'Arnaud
4          Travis d'Arnaud
                ...       
708820       Brenton Doyle
708821      Ezequiel Tovar
708822      Ezequiel Tovar
708823      Ezequiel Tovar
708824    Charlie Blackmon
Name: batter_name, Length: 708540, dtype: object

In [11]:
savant_2024.loc[:, 'batter_name'] = savant_2024['batter_name'].fillna('Unknown Batter')

In [10]:
qualified_num = 162 * 6
shadow_zone_pitches = savant_2024[savant_2024['zone'].isin([11.0, 12.0, 13.0, 14.0])].copy()
catcher_counts = shadow_zone_pitches.groupby('catcher_name').size()
qualified_catchers = catcher_counts[catcher_counts >= qualified_num].index.tolist()
shadow_zone_pitches = shadow_zone_pitches[shadow_zone_pitches['catcher_name'].isin(qualified_catchers)].copy()

In [11]:
useful_catcher_cols = ['catcher_name', 'player_name', 'events', 
                       'description', 'balls', 'strikes', 'zone',
                       'plate_x', 'plate_z', 'sz_top', 'sz_bot', 
                       'woba_value', 'woba_denom','delta_pitcher_run_exp', 
                       'delta_home_win_exp']


catcher_data = shadow_zone_pitches[useful_catcher_cols].reset_index(drop=True).copy()
non_swings = ['ball', 'called_strike']
catcher_data = catcher_data[catcher_data['description'].isin(non_swings)].copy()

In [12]:
catcher_data.loc[:, 'is_called_strike'] = (catcher_data['description']=='called_strike').astype(int).copy()

In [13]:
catcher_data

,catcher_name,player_name,events,description,balls,strikes,zone,plate_x,plate_z,sz_top,sz_bot,woba_value,woba_denom,delta_pitcher_run_exp,delta_home_win_exp,is_called_strike
0,Francisco Alvarez,"Díaz, Edwin",NaN,ball,1,2,14.0,0.75,1.20,3.47,1.52,NaN,NaN,-0.037,0.0,0
1,Francisco Alvarez,"Díaz, Edwin",NaN,ball,0,1,14.0,0.61,1.18,3.47,1.54,NaN,NaN,-0.012,0.0,0
7,Francisco Alvarez,"Díaz, Edwin",NaN,ball,2,2,11.0,-0.40,3.60,3.45,1.59,NaN,NaN,-0.108,0.0,0
8,Francisco Alvarez,"Díaz, Edwin",NaN,ball,1,2,14.0,0.70,1.26,3.61,1.70,NaN,NaN,-0.043,0.0,0
11,Travis d'Arnaud,"Johnson, Pierce",NaN,ball,2,2,11.0,-1.33,2.84,3.13,1.46,NaN,NaN,-0.106,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350758,Logan O'Hoppe,"Fulmer, Carson",NaN,ball,0,1,13.0,-0.22,1.00,3.51,1.53,NaN,NaN,-0.025,0.0,0
350761,Jacob Stallings,"Feltner, Ryan",NaN,ball,1,1,14.0,1.84,0.63,3.44,1.51,NaN,NaN,-0.058,0.0,0
350762,Jacob Stallings,"Feltner, Ryan",NaN,ball,0,0,11.0,-0.86,3.86,3.30,1.49,NaN,NaN,-0.040,0.0,0
350764,Jacob Stallings,"Feltner, Ryan",NaN,ball,1,2,14.0,1.58,0.66,3.48,1.51,NaN,NaN,-0.043,0.0,0


In [14]:
catcher_data['description'].unique()

array(['ball', 'called_strike'], dtype=object)

In [15]:
catcher_data['events'].unique()

array([nan, 'walk', 'strikeout', 'truncated_pa', 'catcher_interf'],
      dtype=object)

In [16]:
no_pitches = catcher_data.groupby('catcher_name')['zone'].size().copy()

In [17]:
strike_zone_pct = catcher_data.groupby(['catcher_name', 'zone'])['is_called_strike'].mean().copy()
strike_zone_pct = strike_zone_pct.reset_index().copy()
strike_zone_pct = strike_zone_pct.merge(no_pitches, how='left', on='catcher_name').rename(columns={'zone_x': 'zone', 'zone_y': 'no_pitches'}).copy()

In [18]:
strike_zone_pct.rename({'zone_x': 'zone', 'zone_y': 'no_pitches'})

,catcher_name,zone,is_called_strike,no_pitches
0,Adley Rutschman,11.0,0.071483,4941
1,Adley Rutschman,12.0,0.067265,4941
2,Adley Rutschman,13.0,0.057511,4941
3,Adley Rutschman,14.0,0.046526,4941
4,Adrian Del Castillo,11.0,0.084906,913
...,...,...,...,...
291,Yan Gomes,14.0,0.024064,1340
292,Yasmani Grandal,11.0,0.031378,3136
293,Yasmani Grandal,12.0,0.112795,3136
294,Yasmani Grandal,13.0,0.081667,3136


In [19]:
strike_zone_pct[strike_zone_pct['zone']==11.0].sort_values(by='is_called_strike', ascending=False)

,catcher_name,zone,is_called_strike,no_pitches
84,Elias Diaz,11.0,0.099237,3515
4,Adrian Del Castillo,11.0,0.084906,913
44,Cal Raleigh,11.0,0.076976,5701
212,Patrick Bailey,11.0,0.072616,5229
0,Adley Rutschman,11.0,0.071483,4941
...,...,...,...,...
140,Joey Bart,11.0,0.027624,3060
80,Dillon Dingler,11.0,0.025723,1105
176,Luis Campusano,11.0,0.021651,3503
208,Omar Narvaez,11.0,0.021605,1153


In [20]:
strike_zone_pct[strike_zone_pct['zone']==12.0].sort_values(by='is_called_strike', ascending=False)

,catcher_name,zone,is_called_strike,no_pitches
117,Hunter Goodman,12.0,0.126697,918
249,Tomas Nido,12.0,0.122727,1944
9,Alejandro Kirk,12.0,0.122124,4275
81,Dillon Dingler,12.0,0.117647,1105
269,Victor Caratini,12.0,0.116883,2655
...,...,...,...,...
229,Riley Adams,12.0,0.054131,1751
257,Tucker Barnhart,12.0,0.051724,1319
5,Adrian Del Castillo,12.0,0.050847,913
273,Will Smith,12.0,0.050052,5616


In [21]:
strike_zone_pct[strike_zone_pct['zone']==13.0].sort_values(by='is_called_strike', ascending=False)

,catcher_name,zone,is_called_strike,no_pitches
78,David Fry,13.0,0.126437,752
102,Gabriel Moreno,13.0,0.096774,3890
26,Austin Hedges,13.0,0.094778,2290
10,Alejandro Kirk,13.0,0.093396,4275
122,Ivan Herrera,13.0,0.089354,2439
...,...,...,...,...
14,Alex Jackson,13.0,0.036923,2231
118,Hunter Goodman,13.0,0.032787,918
226,Rene Pinto,13.0,0.029703,662
202,Mitch Garver,13.0,0.026178,1020


In [22]:
strike_zone_pct[strike_zone_pct['zone']==14.0].sort_values(by='is_called_strike', ascending=False)

,catcher_name,zone,is_called_strike,no_pitches
95,Francisco Alvarez,14.0,0.094074,4266
91,Eric Haase,14.0,0.091483,959
295,Yasmani Grandal,14.0,0.090984,3136
251,Tomas Nido,14.0,0.086721,1944
215,Patrick Bailey,14.0,0.082005,5229
...,...,...,...,...
43,Brian Serven,14.0,0.036111,1017
235,Ryan Jeffers,14.0,0.034107,3704
203,Mitch Garver,14.0,0.031579,1020
7,Adrian Del Castillo,14.0,0.028409,913


In [23]:
delta_run_exp_col = savant_2024.columns.get_loc('delta_run_exp')
new_statcast = savant_2024.iloc[:, delta_run_exp_col+1:].copy()
new_statcast

,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,catcher_name,pitch_uid
0,68.8,7.3,0.059,0.248,88.0,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,1.40,0.96,0.96,17.6,Francisco Alvarez,747139_82_5
1,NaN,NaN,NaN,-0.037,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.14,-0.20,-0.20,23.1,Francisco Alvarez,747139_82_4
2,71.2,8.9,NaN,0.060,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.37,-0.12,-0.12,22.4,Francisco Alvarez,747139_82_3
3,NaN,NaN,NaN,-0.012,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.09,-0.21,-0.21,22.4,Francisco Alvarez,747139_82_2
4,71.7,9.0,NaN,0.040,NaN,-1,-1,0.142,0.142,30,35,30,35,1,4,1.0,1.0,3.0,1.0,2.20,-0.17,-0.17,20.2,Francisco Alvarez,747139_82_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708820,NaN,NaN,NaN,0.041,NaN,0,0,0.537,0.463,30,26,31,26,1,0,6.0,1.0,6.0,1.0,1.16,0.63,0.63,36.7,Logan O'Hoppe,746196_3_1
708821,78.5,7.4,0.064,0.243,88.0,0,0,0.522,0.478,30,22,31,23,1,0,6.0,1.0,6.0,1.0,2.00,1.53,1.53,39.9,Logan O'Hoppe,746196_2_3
708822,NaN,NaN,NaN,-0.024,NaN,0,0,0.522,0.478,30,22,31,23,1,0,6.0,1.0,6.0,1.0,3.10,-0.37,-0.37,37.0,Logan O'Hoppe,746196_2_2
708823,NaN,NaN,NaN,0.037,NaN,0,0,0.522,0.478,30,22,31,23,1,0,6.0,1.0,6.0,1.0,2.02,1.30,1.30,34.7,Logan O'Hoppe,746196_2_1
